In [6]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


In [3]:
df_eng_old_projects = pd.read_csv("G:\ESA_downloads\download_Bingjie_Mar262021\ESA_website_ENG_old_projects.csv")
df_eng_new_projects = pd.read_csv("G:\ESA_downloads\phase_2_download_files\ESA_downloads\ESA_website_ENG_new_projects.csv")

df_fra_old_projects = pd.read_csv("G:\ESA_downloads\download_Bingjie_May062021_fra\ESA_website_FRA.csv")
df_fra_new_projects = pd.read_csv("G:\ESA_downloads\phase_2_download_files\ESA_downloads_fra\ESA_website_FRA_new_projects.csv")

df_eng_all_projects = pd.read_csv("G:\ESA_downloads\ESA_with_VCs\ESA_website_ENG.csv")

C:\Users\ThibJacq\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (52) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
df_eng_old_projects.head()

,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,ID,Project Download Path,Table Download Path,Good Quality
0,Figure 13.1-1 EnCana Ekwan Pipeline,Figure,Application to Construct and Operate Ekwan Pip...,Ekwan,2003-03-17,EnCana Ekwan Pipeline Inc.,Gas,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"Alberta, British Columbia",GH-1-2003,AXYS Environmental Consulting Ltd.,Operating,GC-108,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,"Section 13.1: Introduction, Section 13.1: Proj...",1.0,"Land, Air, Vegetation",26,107.0,1.41,Yes,15151,NaN,NaN,NaN
1,Figure 13.3-1 CEA Framework,Figure,Application to Construct and Operate Ekwan Pip...,Ekwan,2003-03-17,EnCana Ekwan Pipeline Inc.,Gas,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"Alberta, British Columbia",GH-1-2003,AXYS Environmental Consulting Ltd.,Operating,GC-108,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,"Section 13.1: Introduction, Section 13.1: Proj...",1.0,"Land, Air, Vegetation",41,107.0,1.41,Yes,15152,NaN,NaN,NaN
2,Figure 13.4-1 Temperature Normals Measured at ...,Figure,Application to Construct and Operate Ekwan Pip...,Ekwan,2003-03-17,EnCana Ekwan Pipeline Inc.,Gas,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"Alberta, British Columbia",GH-1-2003,AXYS Environmental Consulting Ltd.,Operating,GC-108,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,"Section 13.1: Introduction, Section 13.1: Proj...",1.0,"Land, Air, Vegetation",44,107.0,1.41,Yes,15153,NaN,NaN,NaN
3,Figure 13.4-2 Mean Monthly Rainfall and Number...,Figure,Application to Construct and Operate Ekwan Pip...,Ekwan,2003-03-17,EnCana Ekwan Pipeline Inc.,Gas,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"Alberta, British Columbia",GH-1-2003,AXYS Environmental Consulting Ltd.,Operating,GC-108,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,"Section 13.1: Introduction, Section 13.1: Proj...",1.0,"Land, Air, Vegetation",45,107.0,1.41,Yes,15154,NaN,NaN,NaN
4,Figure 13.4-3 Mean Monthly Snowfall and Number...,Figure,Application to Construct and Operate Ekwan Pip...,Ekwan,2003-03-17,EnCana Ekwan Pipeline Inc.,Gas,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"Alberta, British Columbia",GH-1-2003,AXYS Environmental Consulting Ltd.,Operating,GC-108,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,"Section 13.1: Introduction, Section 13.1: Proj...",1.0,"Land, Air, Vegetation",46,107.0,1.41,Yes,15155,NaN,NaN,NaN


In [4]:
def merge_dfs_with_matching_specific_columns(left_df, right_df, left_on, right_on):
    """Merge two dataframes based on common columns and only keep rows where
    the values match.
    """
    merged_df = pd.merge(left_df, right_df, how='inner', left_on=left_on, right_on=right_on)
    return merged_df

left_cols = ['Title', 'Document Number', 'PDF Page Number']
right_cols = ['Title', 'Document Number', 'PDF Page Number']

df_eng_old_merged = merge_dfs_with_matching_specific_columns(df_eng_all_projects, df_eng_old_projects, left_on=left_cols, right_on=right_cols)
df_eng_new_merged = merge_dfs_with_matching_specific_columns(df_eng_all_projects, df_eng_new_projects, left_on=left_cols, right_on=right_cols)

KeyError: 'Data ID'

In [ ]:
df_eng_new_merged

In [ ]:
df_fra_old_merged = pd.merge(df_eng_old_merged, df_fra_old_projects, how='inner', on=['ID'])
df_fra_new_merged = pd.merge(df_eng_new_merged, df_fra_new_projects, how='inner', on=['ID'])